In [14]:
import sys
!{sys.executable} -m pip install gym



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import gym
import numpy as np
from gym import spaces

class ThermostatEnvironment(gym.Env):
    def __init__(self):
        self.action.space = spaces.Discrete(3) #increase, decrease, maintain
        self.observation_space = spaces.Box(low=18, high=np.array([25]), dtype=np.float32) #min temp = 18, max temp = 25
        self.set_point = 18
        self.temperature = 18
        self.max_temp = 25
        self.min_temp = 18
        self.reward = 0
        self.done = False
    
    def step(self, action):
        if action == 0:
            self.temperature += 1 #increase temperature
            self.reward = 2 #reward for increasing temperature
        elif action == 1:
            self.temperature -= 1 #decrease temperature
            self.reward = -1  #penalty for decreasing temperature
        else:
            pass
        if self.temperature < self.min_temp:
            self.temperature = self.min_temp
        elif self.temperature > self.max_temp:
            self.temperature = self.max_temp
        if self.temperature == self.set_point:
            self.reward = 1
        else:
            self.reward = 0
        return self.temperature, self.reward, self.done, {}

    def reset(self):
        self.temperature = 18
        return self.temperature
    
    def action_space(self):
        return self.action_space

In [8]:
import tensorflow as tf

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque

class DQN(nn.Module):
    def __init__(self, input_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_size, 10)
        self.fc2 = nn.Linear(10, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [17]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

class DQN:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self.build_model()

    # neural network model 

    #return action based on state
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            act_values = self.model.predict(state)
        return np.argmax(act_values.cpu().data.numpy())
    
    # remember experience
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            state = torch.FloatTensor(state).unsqueeze(0)
            next_state = torch.FloatTensor(next_state).unsqueeze(0)
            target = reward
            if not done:
                with torch.no_grad():
                    target = (reward + self.gamma * torch.max(self.model(next_state)[0])).item()
            target_f = self.model(state)
            target_f[0][action] = target
            self.optimizer.zero_grad()
            loss = nn.MSELoss()(target_f, self.model(state))
            loss.backward()
            self.optimizer.step()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [22]:
env = ThermostatEnvironment()
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQN(state_size, action_size)
episodes = 500

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}".format(e, episodes, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

AttributeError: 'ThermostatEnvironment' object has no attribute 'action'